# OOP (2024 Fall) 실습02: 캡슐화 원리가 적용된 방향그래프
- 이름: 연유경
- 학번: 20232932

In [10]:
import javax.imageio.*;

var mmgraph = """
graph LR;
    서울 --> 원주 --> 평창 --> 강릉
    원주 --> 안동
    서울 --> 천안 --> 대전 & 공주
    대전 --> 대구 --> 부산
    공주 --> 익산 --> 광주 --> 목포
    대전 --> 익산 --> 전주 --> 여수
""";

display(ImageIO.read(new URL("https://mermaid.ink/img/"+Base64.getEncoder().encodeToString(mmgraph.getBytes()))));

CompilerException: 

## 방향그래프 레코드
노드가 String인 인접리스트 표현을 구성요소로 포함하는 레코드의 정의를 캡슐화 원리가 잘 지켜지도록 수정/완성해 보라.

adjList는 실습01에서 작성했던 routeMap으로 초기화한다고 보면 된다.

In [25]:
import java.util.*;

public record Graph(Map<String, List<String>> adjList) {
    // 생성자에서 깊은 복사로 캡슐화 유지
    public Graph(Map<String, List<String>> adjList) {
        Map<String, List<String>> deepCopy = new HashMap<>();
        for (Map.Entry<String, List<String>> entry : adjList.entrySet()) {
            deepCopy.put(entry.getKey(), List.copyOf(entry.getValue()));
        }
        this.adjList = Map.copyOf(deepCopy);
    }

    // orig 노드로부터 인접 노드 리스트 반환 (없는 경우 null)
    public List<String> adjFrom(String orig) {
        return adjList.get(orig); // 기본적으로 null 반환
    }

    // 선택사항: 경로 존재 여부 확인
    public boolean hasPath(String orig, String dest) {
        if (!adjList.containsKey(orig)) return false;

        return dfs(orig, dest, new HashSet<>());
    }

    // 깊이 우선 탐색 (DFS) 구현
    private boolean dfs(String current, String dest, Set<String> visited) {
        if (current.equals(dest)) return true; // 목적지 도달 시 true

        visited.add(current); // 현재 노드 방문 기록

        // 인접 노드 탐색
        for (String neighbor : adjList.getOrDefault(current, Collections.emptyList())) {
            if (visited.add(neighbor)) { // 방문하지 않은 노드만 탐색
                if (dfs(neighbor, dest, visited)) return true;
            }
        }

        return false; // 경로 없을 시 false 반환
    }
}


#### 코드에서 어떤 부분을 신경써서 캡슐화 원리가 잘 지켜지도록 작성했는지 되도록 짧게 설명하라. (200자 넘지 않게)


캡슐화 원리를 지키기 위해 adjList의 깊은 복사를 사용하여 외부에서 원본 그래프 구조가 수정되지 않도록 방어적 복사를 적용했습니다. 생성자에서 입력받은 맵과 리스트 모두 불변으로 복사하여 외부에서의 직접적인 수정이 불가능하게 했습니다.





#### 그래프 객체 생성 및 활용
실습01에서처럼 위의 하행선 철도 노선도를 자바의 Map과 List를 활용한 인접리스트(adjacencty list) 데이터 구조로 옮겨 `Graph` 레코드의 인스턴스를 생성해 `routeGraph` 변수에 저장해 보라.

In [26]:
// routeMap은 지난 lab01 내용
var routeMap = new HashMap< String, List<String> >() {{ // modifiable map with modifiable values 
    put("서울", new ArrayList<>(List.of("원주","천안")));
    put("원주", new ArrayList<>(List.of("평창","안동")));  put("안동", new ArrayList<>());
    put("평창", new ArrayList<>(List.of("강릉")));        put("강릉", new ArrayList<>());
    put("천안", new ArrayList<>(List.of("대전","공주")));
    put("대전", new ArrayList<>(List.of("대구","익산")));
    put("대구", new ArrayList<>(List.of("부산")));        put("부산", new ArrayList<>());
    put("공주", new ArrayList<>(List.of("익산")));
    put("익산", new ArrayList<>(List.of("광주","전주")));
    put("광주", new ArrayList<>(List.of("목포")));        put("목포", new ArrayList<>());
    put("전주", new ArrayList<>(List.of("여수")));        put("여수", new ArrayList<>());
}};

System.out.println(routeMap);

{강릉=[], 전주=[여수], 익산=[광주, 전주], 서울=[원주, 천안], 부산=[], 안동=[], 평창=[강릉], 대전=[대구, 익산], 공주=[익산], 원주=[평창, 안동], 목포=[], 여수=[], 천안=[대전, 공주], 대구=[부산], 광주=[목포]}


In [27]:
var routeGraph = new Graph(routeMap);

System.out.println(routeGraph);

Graph[adjList={평창=[강릉], 안동=[], 여수=[], 목포=[], 대전=[대구, 익산], 익산=[광주, 전주], 부산=[], 대구=[부산], 천안=[대전, 공주], 서울=[원주, 천안], 원주=[평창, 안동], 강릉=[], 전주=[여수], 광주=[목포], 공주=[익산]}]


그리고 adjFrom 메소드를 호출했을 때 크기가 0, 1, 2인 리스트를 리턴하는 경우 및 null을 리턴하는 경우를 작성해 보라.

In [28]:
System.out.println( routeGraph.adjFrom("서울") );

[원주, 천안]


In [33]:
System.out.println( routeGraph.adjFrom("평창") );

[강릉]


In [30]:
System.out.println( routeGraph.adjFrom("1") );

null


In [31]:
System.out.println( routeGraph.adjFrom("강릉") );

[]
